# Lib

In [1]:
%load_ext autoreload
%autoreload 2

import pandas as pd

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 10000)


import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import sys
import os
import glob
import joblib

if sys.platform=='win32':
    sys.path.insert(0,".\..\src")
        
elif sys.platform=='linux':
    sys.path.insert(0,"./../src")

from utils import utils_ml, parameters


In [2]:
if sys.platform=='win32':
    !dir .\
        
elif sys.platform=='linux':
    !ls ./

 O volume na unidade C � OS
 O N�mero de S�rie do Volume � C498-8BF5

 Pasta de c:\Users\caanp\OneDrive\Documents\repositories\ML-anti-fraud\notebooks

18/11/2022  19:16    <DIR>          .
18/11/2022  16:18    <DIR>          ..
10/09/2022  23:28    <DIR>          .ipynb_checkpoints
18/11/2022  16:18         1.047.962 1.EA.ipynb
18/11/2022  20:25            67.359 2.Feature_engineering.ipynb
18/11/2022  20:26            20.594 3.Stability_var.ipynb
18/11/2022  19:08           131.212 4.Modeling.ipynb
18/11/2022  20:25    <DIR>          catboost_info
               4 arquivo(s)      1.267.127 bytes
               4 pasta(s)   342.609.620.992 bytes dispon�veis


# Reading

In [3]:
## para saber en que directório uno se encuentra
#
if sys.platform=='win32':
    !echo %cd%
        
elif sys.platform=='linux':
    !pwd

c:\Users\caanp\OneDrive\Documents\repositories\ML-anti-fraud\notebooks


In [4]:
df = pd.read_parquet(".\\..\data\dados_transformed.parquet")
df['data'] = pd.to_datetime(df['data'])

In [23]:
df.groupby(pd.Grouper(key='data', freq="W")).agg(

    total_um = ("fraude",lambda x: sum(x) ),
    total_zero = ("fraude",lambda x: sum( (x+1)%2 ) ) 


)

,total_um,total_zero
data,,
2020-03-08,124,2714
2020-03-15,1184,23339
2020-03-22,1149,24050
2020-03-29,1142,18927
2020-04-05,1008,16426
2020-04-12,1322,23121
2020-04-19,1087,24245
2020-04-26,484,9678


In [6]:
gb = df.groupby(pd.Grouper(key='data', freq="W"))

dff_with_week = []

for week, df_week in gb:

    df_week['week'] = week

    dff_with_week.append(df_week)


dff_week = pd.concat(dff_with_week)

C:\Users\caanp\AppData\Local\Temp\ipykernel_23904\26454910.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_week['week'] = week
C:\Users\caanp\AppData\Local\Temp\ipykernel_23904\26454910.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_week['week'] = week
C:\Users\caanp\AppData\Local\Temp\ipykernel_23904\26454910.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documen

In [36]:
dff_week[['data','week']]

,data,week
29,2020-03-08,2020-03-08
126,2020-03-08,2020-03-08
261,2020-03-08,2020-03-08
351,2020-03-08,2020-03-08
386,2020-03-08,2020-03-08
...,...,...
149918,2020-04-21,2020-04-26
149944,2020-04-21,2020-04-26
149953,2020-04-21,2020-04-26
149972,2020-04-21,2020-04-26


In [14]:
df['week'] = gb["data"].transform("first")

In [7]:
dff_week.groupby("week").agg( 

    set_data = ("data",lambda x: list(np.unique(x)) ),
     total = ("data",lambda x: len(list(np.unique(x))) ),
     total_um = ("fraude",lambda x: sum(x) ),
    total_zero = ("fraude",lambda x: sum( (x+1)%2 ) )
    
     )

,set_data,total,total_um,total_zero
week,,,,
2020-03-08,[2020-03-08T00:00:00.000000000],1,124,2714
2020-03-15,"[2020-03-09T00:00:00.000000000, 2020-03-10T00:...",7,1184,23339
2020-03-22,"[2020-03-16T00:00:00.000000000, 2020-03-17T00:...",7,1149,24050
2020-03-29,"[2020-03-23T00:00:00.000000000, 2020-03-24T00:...",7,1142,18927
2020-04-05,"[2020-03-30T00:00:00.000000000, 2020-03-31T00:...",7,1008,16426
2020-04-12,"[2020-04-06T00:00:00.000000000, 2020-04-07T00:...",7,1322,23121
2020-04-19,"[2020-04-13T00:00:00.000000000, 2020-04-14T00:...",7,1087,24245
2020-04-26,"[2020-04-20T00:00:00.000000000, 2020-04-21T00:...",2,484,9678


In [275]:
colunas_orig_and_week = utils_ml.exclude_words( dff_week.columns.tolist(), ["cat","bin","WoE","produto","fecha","data","hora","week","fraude"] ) + ['categoria_produto']
# colunas_orig_and_week_bin = utils_ml.exclude_words(colunas_orig_and_week_bin , ['categoria'] ) + ['categoria_produto_cat']
weeks = dff_week['week'].unique()

len(colunas_orig_and_week), len(weeks)

(17, 8)

In [264]:
colunas_orig_and_week

['a_int',
 'b_float',
 'c_float',
 'd_float',
 'e_float',
 'f_float',
 'pais',
 'h_float',
 'k_float',
 'l_float',
 'm_float',
 'n_boolean',
 'o_obj',
 'p_boolean',
 'monto',
 'score',
 'categoria_produto']

In [266]:
label = "fraude"
dtype_ = 'categorical'
df_woe_ivs_ls = []

for n,variable in enumerate(colunas_orig_and_week, start=1):

    print(f"{n}/{len(colunas_orig_and_week)}",variable)

    ## carregando o modelo pretreinado de
    optb = joblib.load(f".\..\src\\features\\pipeline_binning__{variable}__.pkl")

    for week in weeks:

        df_filtred = dff_week[dff_week['week']==week][[variable,label]]
        df_filtred = utils_ml.encode_woe_var_transform(df_filtred, variable, optb )

        ## computting the WOE and IV to variable transformed with _cat in the end.
        df_iv = utils_ml.WOE_IV(df = df_filtred  , variable = f"{variable}_bins", dtype_ = dtype_ , label = label, monotonic= True, optimal_binning = False)
        df_iv['week'] = week


        df_woe_ivs_ls.append(df_iv)

df_woe_ivs = pd.concat(df_woe_ivs_ls).sort_values("IV_var", ascending=False)

1/17 a_int
2/17 b_float
3/17 c_float
4/17 d_float
5/17 e_float
6/17 f_float
7/17 pais
8/17 h_float
9/17 k_float
10/17 l_float
11/17 m_float
12/17 n_boolean
13/17 o_obj
14/17 p_boolean
15/17 monto
16/17 score
17/17 categoria_produto


In [274]:
df_woe_ivs['Bin'] = df_woe_ivs['Bin'].astype("string")
df_woe_ivs.query("Bin!='Missing' and Bin!='Special'").to_parquet(".\..\data\\iv_woe_by_week.parquet")

## Data to explore in powerBI

# Requirements

In [110]:
path_root = parameters.root
!pip list --format=freeze > $path_root/requirements.txt

5697.85s - pydevd: Sending message related to process being replaced timed-out after 5 seconds
